In [1]:
# Import dependencies
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

from pprint import pprint

app = Flask(__name__)

In [2]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///../db/developers.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Survey = Base.classes.survey

/Users/alexstanley/anaconda3/envs/python36/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
# find devtypes, languageworkedwith, databaseworkedwith, frameworkWorkededWith
s = [Survey.Respondent,
     Survey.DevType,
     Survey.LanguageWorkedWith,
     Survey.ConvertedSalary
    ]

radarResults = db.session.query(*s).filter(Survey.ConvertedSalary != '').all()

In [4]:
# put results into pandas
df = pd.DataFrame(radarResults)
df['ConvertedSalary'] = pd.to_numeric(df['ConvertedSalary'])
df.head()

Respondent                                            DevType  \
0        2999  Data or business analyst;Educator or academic ...   
1        2943  Back-end developer;Database administrator;Desk...   
2        3203                                 Back-end developer   
3        3010                                   Mobile developer   
4        3067                                   Mobile developer   

                            LanguageWorkedWith  ConvertedSalary  
0                         C;C++;Java;SQL;Swift             8400  
1                 C++;Java;JavaScript;HTML;CSS             4980  
2               C++;Java;Python;SQL;Bash/Shell            36716  
3  C;C#;Java;Objective-C;Rust;Swift;TypeScript            51408  
4               C;JavaScript;Objective-C;Swift            10175

In [5]:
# explode the devtype column so there's only one devtype for each row
expdev_df = pd.DataFrame(df.DevType.str.split(';').tolist(), index=[df.Respondent, df.LanguageWorkedWith, df.ConvertedSalary]).stack()

expdev_df = expdev_df.reset_index()

expdev_df = expdev_df.rename(columns={0:'DevType'})
expdev_df = expdev_df.drop(columns='level_3')

expdev_df.head()

Respondent    LanguageWorkedWith  ConvertedSalary  \
0        2999  C;C++;Java;SQL;Swift             8400   
1        2999  C;C++;Java;SQL;Swift             8400   
2        2999  C;C++;Java;SQL;Swift             8400   
3        2999  C;C++;Java;SQL;Swift             8400   
4        2999  C;C++;Java;SQL;Swift             8400   

                           DevType  
0         Data or business analyst  
1  Educator or academic researcher  
2              Engineering manager  
3                 Mobile developer  
4             System administrator

In [6]:
expdev_df['LanguageList'] = expdev_df.LanguageWorkedWith.str.split(';')
expdev_df.head()

Respondent    LanguageWorkedWith  ConvertedSalary  \
0        2999  C;C++;Java;SQL;Swift             8400   
1        2999  C;C++;Java;SQL;Swift             8400   
2        2999  C;C++;Java;SQL;Swift             8400   
3        2999  C;C++;Java;SQL;Swift             8400   
4        2999  C;C++;Java;SQL;Swift             8400   

                           DevType                LanguageList  
0         Data or business analyst  [C, C++, Java, SQL, Swift]  
1  Educator or academic researcher  [C, C++, Java, SQL, Swift]  
2              Engineering manager  [C, C++, Java, SQL, Swift]  
3                 Mobile developer  [C, C++, Java, SQL, Swift]  
4             System administrator  [C, C++, Java, SQL, Swift]

In [7]:
# hard code top 10 languages from stack ooverflow website
topLang = ['JavaScript', 'HTML', 'CSS', 'SQL', 'Java', 'Bash/Shell', 'Python', 'C#', 'PHP', 'C++']

In [8]:
# Test case for one language
# Check if language is in each entries list of known languagesLanguage List. If yes, return 1, otherwise, return 0
# expdev_df['Python'] = np.where(expdev_df.LanguageList.apply(lambda x: 'Python' in x), 1, 0)

In [9]:
expdev_df.head()

Respondent    LanguageWorkedWith  ConvertedSalary  \
0        2999  C;C++;Java;SQL;Swift             8400   
1        2999  C;C++;Java;SQL;Swift             8400   
2        2999  C;C++;Java;SQL;Swift             8400   
3        2999  C;C++;Java;SQL;Swift             8400   
4        2999  C;C++;Java;SQL;Swift             8400   

                           DevType                LanguageList  
0         Data or business analyst  [C, C++, Java, SQL, Swift]  
1  Educator or academic researcher  [C, C++, Java, SQL, Swift]  
2              Engineering manager  [C, C++, Java, SQL, Swift]  
3                 Mobile developer  [C, C++, Java, SQL, Swift]  
4             System administrator  [C, C++, Java, SQL, Swift]

In [10]:
# Loop through each language and see if each respondent knows the language
for language in topLang:
    
    cName = 'salary' + language
    # dynamically create new column headers
    print('Checking for: ' + language + '...')        
    
    # Create new column to check if the respondent knows a language. If the languege is in the list of known languages, it gets 1, else it gets 0
    expdev_df[language] = np.where(expdev_df.LanguageList.apply(lambda x: language in x), 1, 0)
    expdev_df[cName] = np.where(expdev_df.LanguageList.apply(lambda x: language in x), expdev_df['ConvertedSalary'], 0)

    print(f'{language} done!')


Checking for: JavaScript...
JavaScript done!
Checking for: HTML...
HTML done!
Checking for: CSS...
CSS done!
Checking for: SQL...
SQL done!
Checking for: Java...
Java done!
Checking for: Bash/Shell...
Bash/Shell done!
Checking for: Python...
Python done!
Checking for: C#...
C# done!
Checking for: PHP...
PHP done!
Checking for: C++...
C++ done!


In [11]:
expdev_df.head()

Respondent    LanguageWorkedWith  ConvertedSalary  \
0        2999  C;C++;Java;SQL;Swift             8400   
1        2999  C;C++;Java;SQL;Swift             8400   
2        2999  C;C++;Java;SQL;Swift             8400   
3        2999  C;C++;Java;SQL;Swift             8400   
4        2999  C;C++;Java;SQL;Swift             8400   

                           DevType                LanguageList  JavaScript  \
0         Data or business analyst  [C, C++, Java, SQL, Swift]           0   
1  Educator or academic researcher  [C, C++, Java, SQL, Swift]           0   
2              Engineering manager  [C, C++, Java, SQL, Swift]           0   
3                 Mobile developer  [C, C++, Java, SQL, Swift]           0   
4             System administrator  [C, C++, Java, SQL, Swift]           0   

   salaryJavaScript  HTML  salaryHTML  CSS    ...      Bash/Shell  \
0                 0     0           0    0    ...               0   
1                 0     0           0    0    ...               0   
2                 0     0           0    0    ...               0   
3                 0     0           0    0    ...               0   
4                 0     0           0    0    ...               0   

   salaryBash/Shell  Python  salaryPython  C#  salaryC#  PHP  salaryPHP  C++  \
0                 0       0             0   0         0    0          0    1   
1                 0       0             0   0         0    0          0    1   
2                 0       0             0   0         0    0          0    1   
3                 0       0             0   0         0    0          0    1   
4                 0       0             0   0         0    0          0    1   

   salaryC++  
0       8400  
1       8400  
2       8400  
3       8400  
4       8400  

[5 rows x 25 columns]

In [12]:
# Aggregate by sum. This will get both the number of coders for a language but also the total salary for that language
totals = expdev_df.groupby('DevType').sum()
# drop unnecessary colums
totals = totals.drop(columns=['Respondent', 'ConvertedSalary'], index='')

totals

JavaScript  salaryJavaScript  \
DevType                                                                       
Back-end developer                                  22051        2129999243   
C-suite executive (CEO, CTO, etc.)                   1324         158907493   
Data or business analyst                             2635         292922001   
Data scientist or machine learning specialist        2009         209233855   
Database administrator                               5695         550811485   
Designer                                             4337         397038051   
Desktop or enterprise applications developer         6034         603072497   
DevOps specialist                                    4645         545170941   
Educator or academic researcher                      1067          96849508   
Embedded applications or devices developer           1415         132333853   
Engineering manager                                  2226         300099134   
Front-end developer                                 16342        1572522689   
Full-stack developer                                22089        2225687877   
Game or graphics developer                           1328         112228385   
Marketing or sales professional                       370          40371325   
Mobile developer                                     6268         548066257   
Product manager                                      1809         204989994   
QA or test developer                                 2642         257158781   
Student                                              3104         146274703   
System administrator                                 4479         453497553   

                                                HTML  salaryHTML    CSS  \
DevType                                                                   
Back-end developer                             20743  1946176392  19880   
C-suite executive (CEO, CTO, etc.)              1282   152554122   1242   
Data or business analyst                        2722   296474863   2546   
Data scientist or machine learning specialist   2024   204975597   1850   
Database administrator                          5704   535535383   5522   
Designer                                        4470   398948565   4369   
Desktop or enterprise applications developer    5979   591296192   5708   
DevOps specialist                               4312   493722169   4123   
Educator or academic researcher                 1120    97814267   1065   
Embedded applications or devices developer      1362   130394796   1245   
Engineering manager                             2103   271505029   2013   
Front-end developer                            15884  1496376569  15718   
Full-stack developer                           20997  2074247754  20531   
Game or graphics developer                      1326   103055722   1269   
Marketing or sales professional                  392    41038461    378   
Mobile developer                                6017   512287901   5829   
Product manager                                 1807   205022761   1744   
QA or test developer                            2622   250097186   2507   
Student                                         3346   143128950   3188   
System administrator                            4474   433013838   4299   

                                                salaryCSS    SQL   salarySQL  \
DevType                                                                        
Back-end developer                             1879105409  19216  1864067573   
C-suite executive (CEO, CTO, etc.)              146526190   1135   138587134   
Data or business analyst                        280882905   3135   346084270   
Data scientist or machine learning specialist   189151952   2230   233427206   
Database administrator                          519486455   5914   571388003   
Designer                                        388271379   3559   319259519   
Desktop or enterp

In [13]:
counts = totals[topLang]

counts

JavaScript   HTML    CSS  \
DevType                                                                   
Back-end developer                                  22051  20743  19880   
C-suite executive (CEO, CTO, etc.)                   1324   1282   1242   
Data or business analyst                             2635   2722   2546   
Data scientist or machine learning specialist        2009   2024   1850   
Database administrator                               5695   5704   5522   
Designer                                             4337   4470   4369   
Desktop or enterprise applications developer         6034   5979   5708   
DevOps specialist                                    4645   4312   4123   
Educator or academic researcher                      1067   1120   1065   
Embedded applications or devices developer           1415   1362   1245   
Engineering manager                                  2226   2103   2013   
Front-end developer                                 16342  15884  15718   
Full-stack developer                                22089  20997  20531   
Game or graphics developer                           1328   1326   1269   
Marketing or sales professional                       370    392    378   
Mobile developer                                     6268   6017   5829   
Product manager                                      1809   1807   1744   
QA or test developer                                 2642   2622   2507   
Student                                              3104   3346   3188   
System administrator                                 4479   4474   4299   

                                                 SQL   Java  Bash/Shell  \
DevType                                                                   
Back-end developer                             19216  13126       13335   
C-suite executive (CEO, CTO, etc.)              1135    643         876   
Data or business analyst                        3135   1506        1763   
Data scientist or machine learning specialist   2230   1560        2020   
Database administrator                          5914   2842        3586   
Designer                                        3559   2301        2201   
Desktop or enterprise applications developer    5905   3818        3554   
DevOps specialist                               4034   2698        4187   
Educator or academic researcher                  918    768         949   
Embedded applications or devices developer      1147   1126        1510   
Engineering manager                             1923   1331        1651   
Front-end developer                            11628   7397        7670   
Full-stack developer                           17252  10675       11502   
Game or graphics developer                       931    960         869   
Marketing or sales professional                  305    166         197   
Mobile developer                                4784   5458        3289   
Product manager                                 1619    935        1148   
QA or test developer                            2402   1706        1865   
Student                                         2553   2695        2003   
System administrator                            4108   2389        3791   

                                               Python     C#   PHP   C++  
DevType                                                                   
Back-end developer                              11218  11256  9595  6301  
C-suite executive (CEO, CTO, etc.)                659    588   722   416  
Data or business analyst                         2001   1559  1166   828  
Data scientist or machine learning specialist    2871    917   698  1229  
Database administrator                           2867   3040  3332  1577  
Designer                                         1874   2213  2271  1425  
Desktop or enterprise applications developer     2962   5337  2180  2963  
DevOps specialist                                3182   2086  2003  12

In [14]:
# empty list to hold column names
salaryList = []

# concat salery to each language so it matches the prev for loop
for l in topLang:
    c = 'salary' + l
    salaryList.append(c)

sals = totals[salaryList]
sals.columns = topLang

sals

JavaScript        HTML  \
DevType                                                                 
Back-end developer                             2129999243  1946176392   
C-suite executive (CEO, CTO, etc.)              158907493   152554122   
Data or business analyst                        292922001   296474863   
Data scientist or machine learning specialist   209233855   204975597   
Database administrator                          550811485   535535383   
Designer                                        397038051   398948565   
Desktop or enterprise applications developer    603072497   591296192   
DevOps specialist                               545170941   493722169   
Educator or academic researcher                  96849508    97814267   
Embedded applications or devices developer      132333853   130394796   
Engineering manager                             300099134   271505029   
Front-end developer                            1572522689  1496376569   
Full-stack developer                           2225687877  2074247754   
Game or graphics developer                      112228385   103055722   
Marketing or sales professional                  40371325    41038461   
Mobile developer                                548066257   512287901   
Product manager                                 204989994   205022761   
QA or test developer                            257158781   250097186   
Student                                         146274703   143128950   
System administrator                            453497553   433013838   

                                                      CSS         SQL  \
DevType                                                                 
Back-end developer                             1879105409  1864067573   
C-suite executive (CEO, CTO, etc.)              146526190   138587134   
Data or business analyst                        280882905   346084270   
Data scientist or machine learning specialist   189151952   233427206   
Database administrator                          519486455   571388003   
Designer                                        388271379   319259519   
Desktop or enterprise applications developer    567159450   574474352   
DevOps specialist                               474489543   476192349   
Educator or academic researcher                  90302814    78284438   
Embedded applications or devices developer      120774231   115646046   
Engineering manager                             258387669   245782148   
Front-end developer                            1487396329  1096657823   
Full-stack developer                           2047083543  1733004454   
Game or graphics developer                       97889904    73520447   
Marketing or sales professional                  39323068    32790209   
Mobile developer                                503112506   412552307   
Product manager                                 197523953   177045732   
QA or test developer                            238489557   226849087   
Student                                         139967809   111088331   
System administrator                            414199997   417804132   

                                                     Java  Bash/Shell  \
DevType                                                                 
Back-end developer                             1199814225  1430948794   
C-suite executive (CEO, CTO, etc.)               70597188   116006787   
Data or business analyst                        149894746   213549293   
Data scientist or machine learning specialist   151900581   221622479   
Database administrator                          248567294   389061142   
Designer                                        174735874   224543626   
Desktop or enterprise applications developer    342546650   364862446   
DevOps specialist                               307841409   495283252   
Educator or academic researcher                  54327625    86829601   
Embedded applications or devic

In [15]:
salLangDev = sals/counts

salLangDev

JavaScript           HTML  \
DevType                                                                       
Back-end developer                              96594.224434   93823.284578   
C-suite executive (CEO, CTO, etc.)             120020.765106  118996.975039   
Data or business analyst                       111165.844782  108918.024614   
Data scientist or machine learning specialist  104148.260329  101272.528162   
Database administrator                          96718.434592   93887.689867   
Designer                                        91546.703021   89250.238255   
Desktop or enterprise applications developer    99945.723732   98895.499582   
DevOps specialist                              117367.263940  114499.575371   
Educator or academic researcher                 90768.048735   87334.166964   
Embedded applications or devices developer      93522.157597   95737.735683   
Engineering manager                            134815.424079  129103.675226   
Front-end developer                             96225.840717   94206.532926   
Full-stack developer                           100760.010729   98787.815116   
Game or graphics developer                      84509.326054   77719.247360   
Marketing or sales professional                109111.689189  104689.951531   
Mobile developer                                87438.777441   85140.086588   
Product manager                                113316.746269  113460.299391   
QA or test developer                            97334.890613   95384.128909   
Student                                         47124.582152   42776.135684   
System administrator                           101249.732753   96784.496647   

                                                         CSS            SQL  \
DevType                                                                       
Back-end developer                              94522.404879   97006.014415   
C-suite executive (CEO, CTO, etc.)             117975.998390  122103.201762   
Data or business analyst                       110323.214847  110393.706539   
Data scientist or machine learning specialist  102244.298378  104675.877130   
Database administrator                          94075.779609   96616.165539   
Designer                                        88869.622110   89704.838157   
Desktop or enterprise applications developer    99362.202172   97286.088400   
DevOps specialist                              115083.566093  118044.707238   
Educator or academic researcher                 84791.374648   85277.165577   
Embedded applications or devices developer      97007.414458  100824.800349   
Engineering manager                            128359.497765  127811.829433   
Front-end developer                             94630.126543   94311.818283   
Full-stack developer                            99706.957430  100452.379666   
Game or graphics developer                      77139.404255   78969.330827   
Marketing or sales professional                104029.280423  107508.881967   
Mobile developer                                86311.975639   86235.850125   
Product manager                                113259.147362  109354.991970   
QA or test developer                            95129.460311   94441.751457   
Student                                         43904.582497   43512.859773   
System administrator                            96347.987206  101704.998053   

                                                        Java     Bash/Shell  \
DevType                                                                       
Back-end developer                              91407.452766  107307.746082   
C-suite executive (CEO, CTO, etc.)             109793.449456  132427.839041   
Data or business analyst                        99531.703851  121128.356778   
Data scientist or machine learning specialist   97372.167308  109714.098515   
Database administrator                          87462.102041  108494.462354   
Designer           

In [16]:
# Aggregation by mean gives a fraction of dev types who know each skill
SalLangDev = expdev_df.groupby('DevType').mean()
SalLangDev = SalLangDev.drop(index='', columns=['Respondent', 'ConvertedSalary'])

# Round each percent to 2 decimals
SalLangDev = SalLangDev.round(decimals=2)

SalLangDev

JavaScript  HTML   CSS   SQL  \
DevType                                                                       
Back-end developer                                   0.76  0.72  0.69  0.66   
C-suite executive (CEO, CTO, etc.)                   0.82  0.79  0.77  0.70   
Data or business analyst                             0.64  0.67  0.62  0.77   
Data scientist or machine learning specialist        0.53  0.54  0.49  0.59   
Database administrator                               0.80  0.80  0.77  0.83   
Designer                                             0.79  0.81  0.80  0.65   
Desktop or enterprise applications developer         0.69  0.68  0.65  0.67   
DevOps specialist                                    0.76  0.71  0.68  0.66   
Educator or academic researcher                      0.62  0.65  0.62  0.53   
Embedded applications or devices developer           0.53  0.51  0.47  0.43   
Engineering manager                                  0.75  0.71  0.68  0.65   
Front-end developer                                  0.88  0.86  0.85  0.63   
Full-stack developer                                 0.87  0.83  0.81  0.68   
Game or graphics developer                           0.68  0.68  0.65  0.48   
Marketing or sales professional                      0.78  0.82  0.79  0.64   
Mobile developer                                     0.68  0.65  0.63  0.52   
Product manager                                      0.79  0.79  0.77  0.71   
QA or test developer                                 0.73  0.73  0.70  0.67   
Student                                              0.66  0.71  0.67  0.54   
System administrator                                 0.78  0.78  0.75  0.72   

                                               Java  Bash/Shell  Python    C#  \
DevType                                                                         
Back-end developer                             0.45        0.46    0.39  0.39   
C-suite executive (CEO, CTO, etc.)             0.40        0.54    0.41  0.36   
Data or business analyst                       0.37        0.43    0.49  0.38   
Data scientist or machine learning specialist  0.41        0.54    0.76  0.24   
Database administrator                         0.40        0.50    0.40  0.43   
Designer                                       0.42        0.40    0.34  0.40   
Desktop or enterprise applications developer   0.43        0.40    0.34  0.61   
DevOps specialist                              0.44        0.69    0.52  0.34   
Educator or academic researcher                0.45        0.55    0.57  0.27   
Embedded applications or devices developer     0.42        0.57    0.57  0.39   
Engineering manager                            0.45        0.56    0.44  0.35   
Front-end developer                            0.40        0.41    0.31  0.39   
Full-stack developer                           0.42        0.45    0.35  0.42   
Game or graphics developer                     0.49        0.45    0.45  0.54   
Marketing or sales professional                0.35        0.41    0.37  0.28   
Mobile developer                               0.59        0.36    0.31  0.37   
Product manager                                0.41        0.50    0.40  0.39   
QA or test developer                           0.47        0.52    0.44  0.39   
Student                                        0.57        0.42    0.52  0.33   
System administrator                           0.42        0.66    0.50  0.34   

                                                PHP   C++  salaryJavaScript  \
DevType                                                                       
Back-end developer                             0.33  0.22          73481.19   
C-suite executive (CEO, CTO, etc.)             0.45  0.26          98091.05   
Data or business analyst                       0.29  0.20          71654.11   
Data scientist or machine learning specialist  0.19  0.33          55602.94   
Database administrator                         0.47

In [13]:
# Convert df to dict to serve as json. Orient by index to make it easily searchable by devtype
SalLangDev_dict = SalLangDev.to_dict(orient='index')

pprint(SalLangDev)

                                               JavaScript      HTML       CSS  \
DevType                                                                         
Back-end developer                               73481.19  67139.63  64825.80   
C-suite executive (CEO, CTO, etc.)               98091.05  94169.21  90448.27   
Data or business analyst                         71654.11  72523.21  68709.13   
Data scientist or machine learning specialist    55602.94  54471.33  50266.26   
Database administrator                           77198.53  75057.52  72808.19   
Designer                                         72320.23  72668.23  70723.38   
Desktop or enterprise applications developer     68476.50  67139.34  64398.71   
DevOps specialist                                89563.16  81110.92  77951.30   
Educator or academic researcher                  56177.21  56736.81  52379.82   
Embedded applications or devices developer       49768.28  49039.04  45420.92   
Engineering manager         